<a href="https://colab.research.google.com/github/greatsong/meet_deep3/blob/main/210206_%EB%B0%91%EB%94%A53_%ED%95%84%EC%82%AC_1%EC%9D%BC%EC%B0%A8_%EC%86%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# step 1

In [1]:
class Variable : 
  def __init__(self, data) :
    self.data = data

In [2]:
import numpy as np
data = np.array(1.0)
x = Variable(data)
print(x.data)

1.0


In [4]:
x.data  = np.array(2)
print(x.data)

2


In [6]:
import numpy as np
x = np.array(1)
x.ndim

0

In [7]:
x = np.array([1,2,3])
x.ndim

1

In [9]:
x = np.array([[1,2,3],
             [4,5,6]])
x.ndim

2

In [ ]:
# step 2

In [10]:
class Function :
  def __call__(self, input) : 
    x = input.data
    y = x ** 2
    output = Variable(y)
    return output

In [11]:
x = Variable(np.array(10))
f = Function()
y = f(x)

In [12]:
print(type(y))
print(y.data)

<class '__main__.Variable'>
100


In [14]:
class Function :
  def __call__(self, input) : 
    x = input.data
    y = self.forward(x)
    output = Variable(y)
    return output
  def forward(self, x) :
    raise NotImplementedError()

In [15]:
class Square(Function) :
  def forward(self, x ) :
    return x ** 2

In [16]:
x = Variable(np.array(10))
f = Square()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
100


In [17]:
# step 3

In [18]:
class Exp(Function) :
  def forward(self, x) :
    return np.exp(x)

In [19]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

print(y.data)

1.648721270700128


In [20]:
# step 4

In [21]:
def numerical_diff(f, x, eps = 1e-4) :
  x0 = Variable(x.data - eps)
  x1 = Variable(x.data + eps)
  y0 = f(x0)
  y1 = f(x1)
  return (y1.data - y0.data) / (2 * eps)

In [22]:
f = Square()
x = Variable(np.array(2))
dy = numerical_diff(f, x)
print(dy)

4.000000000004


In [23]:
def f(x) :
  A = Square()
  B = Exp()
  C = Square()
  return C(B(A(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

3.2974426293330694


In [24]:
# step 5

In [25]:
# step 6

In [26]:
class Variable :
  def __init__(self, data) : 
    self.data = data
    self.grad = None

In [27]:
class Function : 
  def __call__(self, input) : 
    x = input.data
    y = self.forward(x)
    output = Variable(y)
    self.input = input
    return output

  def forward(self, x) :
    raise NotImplementedError()

  def backward(self, gy) :
    raise NotImplementedError()

In [62]:
class Square(Function) :
  def forward(self, x) :
    y = x ** 2
    return y

  def backward(self, gy) :
    x = self.input.data
    gx = 2 * x * gy
    return gx

In [63]:
class Exp(Function) :
  def forward(self, x) :
    y = np.exp(x)
    return y

  def backward(self, gy) : 
    x = self.input.data
    gx = np.exp(x) * gy
    return gx

In [34]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [35]:
y.grad = np.array(1)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)

print(x.grad)

3.297442541400256


In [36]:
# step 7

In [43]:
class Variable :
  def __init__(self, data) : 
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func) :
    self.creator = func

In [48]:
class Function : 
  def __call__(self, input) : 
    x = input.data
    y = self.forward(x)
    output = Variable(y)
    output.set_creator(self)
    self.input = input
    self.output = output
    return output

  def forward(self, x) :
    raise NotImplementedError()

  def backward(self, gy) :
    raise NotImplementedError()

In [55]:
# 그냥 실행하면 오류가 발생 / 왜냐하면 스퀘어, 지수 함수가 업데이트 안되어서!
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [56]:
assert y.creator == C

In [57]:
print(a.creator)

In [58]:
y.grad = np.array(1)

C = y.creator
b = C.input
b.grad = C.backward(y.grad)

In [59]:
B = b.creator
a = B.input
a.grad = B.backward(b.grad)

In [60]:
A = a.creator
x = A.input
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


In [65]:
class Variable :
  def __init__(self, data) : 
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func) :
    self.creator = func

  def backward(self) :
    f = self.creator
    if f is not None : 
      x = f.input
      x.grad = f.backward(self.grad)
      # 중간 과정 확인하기 위한 print 추가
      print(f, x.grad)
      x.backward()

In [66]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1)
y.backward()
print(x.grad)

<__main__.Square object at 0x7f51c8a07390> 2.568050833375483
<__main__.Exp object at 0x7f51c8a071d0> 3.297442541400256
<__main__.Square object at 0x7f51c8a07240> 3.297442541400256
3.297442541400256
